In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import mimetypes
import random
import cv2
from google.colab import drive

In [2]:
# Step 1: Extract the image data
drive.mount('/content/drive')

# Cargar los modelos de IA previamente entrenados
modelo_partes_cuerpo = tf.keras.models.load_model('/content/body_part_model.h5')
modelo_celulas_pulmon = tf.keras.models.load_model('/content/lung_model.h5')
modelo_celulas_colon = tf.keras.models.load_model('/content/colon_model.h5')
modelo_celulas_pecho = tf.keras.models.load_model('/content/breast_model.h5')


Mounted at /content/drive


In [8]:
#Mostrar imágenes visuales y su categoría
#Introducir el path de las imágenes

imagenes_path=['/content/lungaca30.jpeg',
               '/content/lungn30.jpeg',
               '/content/lungscc30.jpeg',
               '/content/colonca30.jpeg',
               '/content/colonn30.jpeg',
              #  '/content/8864_idx5_x1701_y2251_class1.png',
              #  '/content/8864_idx5_x651_y2301_class0.png',
              ]


In [9]:
def preprocesado_imagen(imagen):
    imagen = cv2.imread(imagen)
    imagen_redimensionada = cv2.resize(imagen, (50, 50))
    imagen_rgb = cv2.cvtColor(imagen_redimensionada, cv2.COLOR_BGR2RGB)
    imagen_final=imagen_rgb/255 - 0.5
    return imagen_final

# def preprocesado_imagen(imagen):
#     img = cv2.imread(imagen)
#     if img is None:
#         print("Error al cargar la imagen:", imagen)
#         return None
#     if img.shape[0] < 50 or img.shape[1] < 50:
#         print("La imagen es demasiado pequeña para redimensionar:", imagen)
#         return None
#     img_redimensionada = cv2.resize(img, (50, 50))
#     img_rgb = cv2.cvtColor(img_redimensionada, cv2.COLOR_BGR2RGB)
#     img_normalizada = img_rgb / 255.0 - 0.5
#     return img_normalizada


In [10]:
#Preprocesado
imagenes_preprocesadas=[]
for img in imagenes_path:
    imagenes_preprocesadas.append(preprocesado_imagen(img))

In [11]:
#Clasificador segun la parte del cuerpo

#Extraemos las predicciones
predicciones_cuerpo=modelo_partes_cuerpo.predict(np.array(imagenes_preprocesadas), verbose=0)
parte_cuerpo=[]

for i, celula in enumerate(predicciones_cuerpo):
    celula= np.argmax(celula)
    parte_cuerpo.append(celula)
    if (celula==0): celulas="células mamarias"
    elif (celula==1): celulas="células del colon"
    else: celulas= "células del pulmón"
    print(f'La imagen {i} corresponde a {celulas}')


La imagen 0 corresponde a células del pulmón
La imagen 1 corresponde a células del pulmón
La imagen 2 corresponde a células del pulmón
La imagen 3 corresponde a células del colon
La imagen 4 corresponde a células del colon


In [12]:
#Función para devolver el tipo de célula
def tipo_celula_pecho_colon(tipo):
    threshold = 0.5
    tipo = np.where(tipo >= threshold, 1, 0)
    if (tipo==0): return "Adenocarcionoma (aca)"
    else: return "Células Sanas (n)"

def tipo_celula_pulmon(tipo):
    tipo = np.argmax(tipo)
    if tipo == 0:
        return "Adenocarcionoma (aca)"
    elif tipo == 1:
        return "Células Sanas (n)"
    elif tipo == 2:
        return "Carcinoma de Celulas Escamosas de Pulmón (scc)"
    else:
        return "no identificado"


In [13]:
#Función para devolver la célula
def celula_cuerpo(tipo):
    if (tipo==0): return "mamario"
    elif (tipo==1): return "de colon"
    else: return "de pulmón"

In [14]:
#Modelos de predicción de células benignas y carcionomas
for i, img in enumerate(np.array(imagenes_preprocesadas)):
    cuerpo= celula_cuerpo(parte_cuerpo[i])
    if (cuerpo=='mamario'): #Pecho
        celula= tipo_celula_pecho_colon(modelo_celulas_pecho.predict(np.expand_dims(img, axis=0), verbose=0))
    elif (cuerpo=='de colon'): #Colon
        celula= tipo_celula_pecho_colon(modelo_celulas_colon.predict(np.expand_dims(img, axis=0), verbose=0))
    elif (cuerpo=='de pulmón'): #pulmon
      celula= tipo_celula_pulmon(modelo_celulas_pulmon.predict(np.expand_dims(img, axis=0), verbose=0))
    else:
      continue
    print(f"La imagen {i} corresponde con tejido {cuerpo} que contiene {celula}")

La imagen 0 corresponde con tejido de pulmón que contiene Adenocarcionoma (aca)
La imagen 1 corresponde con tejido de pulmón que contiene Células Sanas (n)
La imagen 2 corresponde con tejido de pulmón que contiene Carcinoma de Celulas Escamosas de Pulmón (scc)
La imagen 3 corresponde con tejido de colon que contiene Adenocarcionoma (aca)
La imagen 4 corresponde con tejido de colon que contiene Células Sanas (n)
